In [1]:
import pandas as pd
import yfinance as yf
from pathlib import Path

In [2]:
def get_usd_twd(start: str, end: str) -> pd.DataFrame:
    """
    下載每日美元兌台幣匯率
    Args:
        start (str): 起始日期，格式"YYYY-MM-DD"
        end (str): 結束日期，格式"YYYY-MM-DD"
    Returns:
        pd.Series: 每日收盤匯率
    """
    df = yf.download("USDTWD=X", start=start, end=end)
    df.to_csv(
        Path.cwd()/"data"/"usd_twd2.csv",
        index=False,
        header=False
    )
    return df["Close"]

In [3]:
path = Path.cwd()
trade_df = pd.read_csv(path/"data"/"trade_record_done.csv", index_col=1)

# 將 index 轉為 datetime 型態，指定 format，這裡根據csv格式是 "%Y/%m/%d"
trade_df.index = pd.to_datetime(trade_df.index, format="%Y/%m/%d")
trade_df.head()

,代號,買入股數,買入價格,賣出股數,賣出價格,股息
交易日期,,,,,,
2023-07-20,006208,30.0,74.82,0.0,0.0,0
2023-07-25,006208,30.0,74.50,0.0,0.0,0
2023-08-07,006208,28.0,74.17,0.0,0.0,0
2023-08-15,006208,29.0,72.17,0.0,0.0,0
2023-08-25,006208,29.0,72.33,0.0,0.0,0


In [4]:
# 計算分析期間（從最早交易日到今天）
start = trade_df.index.min().strftime("%Y-%m-%d")
end = pd.Timestamp.today().strftime("%Y-%m-%d")

start, end

('2021-04-23', '2025-05-03')

In [5]:
usd_twd = get_usd_twd(start, end)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [11]:
# 定義資產分類
asset_codes = ["VOO", "006208", "00687B", "00919"] # ["M-BTC", "BTC", "WBETH", "BNB", "BCH"]

In [7]:
idx = pd.date_range(start, end)
df = pd.DataFrame(index=idx)
df["VOO"] = 0
df["006208"] = 0
df["00687B"] = 0
df["00919"] = 0
df["BTC"] = 0
df["WBETH"] = 0
df["BNB"] = 0
df["BCH"] = 0

df.head()

,VOO,006208,00687B,00919,BTC,WBETH,BNB,BCH
2021-04-23,0,0,0,0,0,0,0,0
2021-04-24,0,0,0,0,0,0,0,0
2021-04-25,0,0,0,0,0,0,0,0
2021-04-26,0,0,0,0,0,0,0,0
2021-04-27,0,0,0,0,0,0,0,0


In [8]:
def calc_daily_holding(trade_df: pd.DataFrame, asset_code: str) -> pd.Series:
    """
    計算每日持有數量（同一天多筆會自動加總）
    Args:
        trade_df: 交易紀錄DataFrame
        asset_code: 資產代號（如"VOO"、"006208"等）
    Returns:
        pd.Series: 每日累積持有數量，index為日期
    """
    # 選出指定資產的所有交易
    df = trade_df[trade_df["代號"] == asset_code].copy()
    # 買入股數 - 賣出股數
    df["淨買入"] = df["買入股數"] - df["賣出股數"]
    
    # 按日期 groupby 並加總（同一天多筆會合併）
    daily = df.groupby(df.index)["淨買入"].sum()

    # 累積持有
    holding = daily.cumsum() # cumulative sum

    return holding

In [13]:
for code in asset_codes:    
    # 計算每日持有數量
    holding = calc_daily_holding(trade_df, code).reindex(idx, method="ffill").fillna(0)

    if code == "VOO":
        # 下載 VOO 價格資料
        price = yf.download(code, start=start, end=end)["Close"].reindex(idx, method="ffill").squeeze()
        ex_rate = usd_twd.reindex(idx, method="ffill").squeeze()

        # 三者相乘
        df[code] = round(holding * price * ex_rate)

    elif code == "006208" or code == "00919":
        price = yf.download(code+".TW", start=start, end=end)["Close"].reindex(idx, method="ffill").squeeze()

        df[code] = round(holding * price)

    elif code == "00687B":
        price = yf.download(code+".TWO", start=start, end=end)["Close"].reindex(idx, method="ffill").squeeze()

        df[code] = round(holding * price)

df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,VOO,006208,00687B,00919,BTC,WBETH,BNB,BCH
2021-04-23,0.0,0.0,0.0,NaN,0,0,0,0
2021-04-24,0.0,0.0,0.0,NaN,0,0,0,0
2021-04-25,0.0,0.0,0.0,NaN,0,0,0,0
2021-04-26,0.0,0.0,0.0,NaN,0,0,0,0
2021-04-27,0.0,0.0,0.0,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...
2025-04-29,177631.0,163546.0,131172.0,21130.0,0,0,0,0
2025-04-30,177147.0,163877.0,130995.0,21060.0,0,0,0,0
2025-05-01,176803.0,163877.0,130995.0,21060.0,0,0,0,0
2025-05-02,172445.0,172074.0,126609.0,21330.0,0,0,0,0
